# CMMA Holding Period Testing

This notebook explores holding period testing for a CMMA-filtered mean reversion strategy that only buys below 2000 and only sells above 2000.

In [ ]:
import sys
import os

# Import our backtester package
sys.path.append(os.path.abspath('../../'))
from backtester import get_price_data, get_vwap
print("Using backtester package")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Try to import seaborn, but don't fail if it's not available
try:
    import seaborn as sns
    print(f"Seaborn version: {sns.__version__}")
    HAS_SEABORN = True
except ImportError:
    print("Seaborn not available, using matplotlib instead")
    HAS_SEABORN = False

## 1. Load Data

First, let's load the Squid_Ink price data and limit it to the first 20,000 timestamps (in-sample data).

In [ ]:
# Load data directly using backtester package
print("Loading price data...")
prices = get_price_data('SQUID_INK', 1)
print(f"Loaded {len(prices)} price data points")

# Limit to first 20,000 timestamps (in-sample data)
in_sample_prices = prices.iloc[:20000]
print(f"Limited to {len(in_sample_prices)} in-sample data points")

# Get VWAP
print("Getting VWAP for SQUID_INK...")
squid_vwap = in_sample_prices['vwap']
print(f"Got VWAP with {len(squid_vwap)} data points")
print(f"VWAP range: {squid_vwap.min()} to {squid_vwap.max()}")

# Calculate returns
returns = squid_vwap.pct_change().dropna()
print(f"Calculated returns with {len(returns)} data points")

## 2. Calculate CMMA (Cumulative Moving Average Momentum)

Let's calculate the CMMA indicator using the same calculation as in the momentum strategy.

In [ ]:
def calculate_cmma(prices, lookback=10):
    """
    Compute the Cumulative Moving Average Momentum (CMMA).
    
    Parameters:
        prices (pd.Series): Series of prices
        lookback (int): Lookback period
        
    Returns:
        pd.Series: CMMA indicator (0-1 range)
    """
    # Calculate raw CMMA
    raw_cmma = (prices - prices.rolling(lookback).mean().shift(1)).divide(np.sqrt(lookback+1)).dropna()
    
    # Normalize using sigmoid function
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    
    cmma = sigmoid(raw_cmma)
    return cmma

# Calculate CMMA with a fixed lookback period
lookback = 20  # Use a fixed lookback period
cmma = calculate_cmma(squid_vwap, lookback)

# Display the first few rows
cmma.head(10)

## 3. Define Filtered CMMA Mean Reversion Strategy

Let's define a strategy that only buys below 2000 and only sells above 2000, using CMMA as a filter.

In [ ]:
def filtered_cmma_strategy(prices, cmma, fair_price, upper_threshold=0.7, lower_threshold=0.3, holding_period=10):
    """
    Implement a filtered CMMA-based mean reversion strategy.
    
    Parameters:
        prices (pd.Series): Series of prices
        cmma (pd.Series): CMMA indicator
        fair_price (float): Fair price to revert to
        upper_threshold (float): Upper threshold for CMMA
        lower_threshold (float): Lower threshold for CMMA
        holding_period (int): Number of periods to hold the position
        
    Returns:
        pd.Series: Portfolio positions (1 for long, -1 for short, 0 for no position)
    """
    # Initialize positions
    positions = pd.Series(0, index=prices.index)
    
    # Get valid indices where CMMA is not NaN
    valid_indices = cmma.dropna().index
    
    # Set positions based on CMMA and fair price
    for time in valid_indices:
        # Get the current price and CMMA value
        current_price = prices.loc[time]
        current_cmma = cmma.loc[time]
        
        # Get the index position
        idx = prices.index.get_loc(time)
        
        # ONLY short above fair price when CMMA is high (strong momentum)
        if current_price > fair_price and current_cmma > upper_threshold:
            # Set short position for holding period
            end_idx = min(idx + holding_period + 1, len(positions))
            positions.iloc[idx+1:end_idx] = -1
        
        # ONLY buy below fair price when CMMA is low (weak momentum)
        elif current_price < fair_price and current_cmma < lower_threshold:
            # Set long position for holding period
            end_idx = min(idx + holding_period + 1, len(positions))
            positions.iloc[idx+1:end_idx] = 1
    
    return positions

## 4. Test Holding Period Parameter

Let's test different holding periods for the CMMA-filtered mean reversion strategy.

In [ ]:
# Define the fair price
FAIR_PRICE = 2000

# Define CMMA thresholds
upper_threshold = 0.7
lower_threshold = 0.3

# Define holding periods to test
holding_periods = [1, 3, 5, 10, 15, 20, 25, 30, 40, 50]

# Initialize results dictionary
results = {}

# Test different holding periods
for hp in holding_periods:
    # Get positions
    positions = filtered_cmma_strategy(squid_vwap, cmma, FAIR_PRICE, upper_threshold, lower_threshold, hp)
    
    # Calculate strategy returns
    strategy_returns = positions.shift(1) * returns
    strategy_returns = strategy_returns.dropna()
    
    # Calculate performance metrics
    total_return = strategy_returns.sum()
    sharpe_ratio = strategy_returns.mean() / strategy_returns.std() * np.sqrt(252)  # Annualized
    win_rate = (strategy_returns > 0).mean()
    
    # Count the number of trades
    num_trades = (positions.diff() != 0).sum()
    
    # Store results
    results[hp] = {
        'Total Return': total_return,
        'Sharpe Ratio': sharpe_ratio,
        'Win Rate': win_rate,
        'Number of Trades': num_trades
    }

# Convert results to DataFrame
results_df = pd.DataFrame(results).T
results_df.index.name = 'Holding Period'

# Sort by Sharpe Ratio
results_df = results_df.sort_values('Sharpe Ratio', ascending=False)

# Display results
results_df

## 5. Visualize Holding Period Results

Let's visualize the results of the holding period testing.

In [ ]:
# Create a DataFrame with holding periods as columns for visualization
metrics = ['Sharpe Ratio', 'Total Return', 'Win Rate', 'Number of Trades']
viz_data = {metric: [] for metric in metrics}

for hp in holding_periods:
    for metric in metrics:
        viz_data[metric].append(results[hp][metric])

viz_df = pd.DataFrame(viz_data, index=holding_periods)
viz_df.index.name = 'Holding Period'

# Plot performance metrics by holding period
if HAS_SEABORN:
    # Use seaborn for better visualization if available
    plt.figure(figsize=(15, 10))
    
    # Plot Sharpe Ratio
    plt.subplot(2, 2, 1)
    sns.lineplot(x=viz_df.index, y=viz_df['Sharpe Ratio'], marker='o')
    plt.title('Sharpe Ratio by Holding Period')
    plt.xlabel('Holding Period')
    plt.ylabel('Sharpe Ratio')
    plt.grid(True)
    
    # Plot Total Return
    plt.subplot(2, 2, 2)
    sns.lineplot(x=viz_df.index, y=viz_df['Total Return'], marker='o')
    plt.title('Total Return by Holding Period')
    plt.xlabel('Holding Period')
    plt.ylabel('Total Return')
    plt.grid(True)
    
    # Plot Win Rate
    plt.subplot(2, 2, 3)
    sns.lineplot(x=viz_df.index, y=viz_df['Win Rate'], marker='o')
    plt.title('Win Rate by Holding Period')
    plt.xlabel('Holding Period')
    plt.ylabel('Win Rate')
    plt.grid(True)
    
    # Plot Number of Trades
    plt.subplot(2, 2, 4)
    sns.lineplot(x=viz_df.index, y=viz_df['Number of Trades'], marker='o')
    plt.title('Number of Trades by Holding Period')
    plt.xlabel('Holding Period')
    plt.ylabel('Number of Trades')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Create a heatmap of all metrics
    plt.figure(figsize=(12, 8))
    sns.heatmap(viz_df.T, annot=True, cmap='viridis', fmt='.2f')
    plt.title('Performance Metrics by Holding Period')
    plt.tight_layout()
    plt.show()
else:
    # Use matplotlib if seaborn is not available
    plt.figure(figsize=(15, 10))
    
    # Plot Sharpe Ratio
    plt.subplot(2, 2, 1)
    plt.plot(viz_df.index, viz_df['Sharpe Ratio'], marker='o')
    plt.title('Sharpe Ratio by Holding Period')
    plt.xlabel('Holding Period')
    plt.ylabel('Sharpe Ratio')
    plt.grid(True)
    
    # Plot Total Return
    plt.subplot(2, 2, 2)
    plt.plot(viz_df.index, viz_df['Total Return'], marker='o')
    plt.title('Total Return by Holding Period')
    plt.xlabel('Holding Period')
    plt.ylabel('Total Return')
    plt.grid(True)
    
    # Plot Win Rate
    plt.subplot(2, 2, 3)
    plt.plot(viz_df.index, viz_df['Win Rate'], marker='o')
    plt.title('Win Rate by Holding Period')
    plt.xlabel('Holding Period')
    plt.ylabel('Win Rate')
    plt.grid(True)
    
    # Plot Number of Trades
    plt.subplot(2, 2, 4)
    plt.plot(viz_df.index, viz_df['Number of Trades'], marker='o')
    plt.title('Number of Trades by Holding Period')
    plt.xlabel('Holding Period')
    plt.ylabel('Number of Trades')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

## 6. Analyze Best Holding Period

Let's analyze the performance of the strategy with the best holding period.

In [ ]:
# Get the best holding period
best_hp = results_df.index[0]
print(f'Best holding period: {best_hp}')

# Get positions for the best holding period
best_positions = filtered_cmma_strategy(squid_vwap, cmma, FAIR_PRICE, upper_threshold, lower_threshold, best_hp)

# Calculate strategy returns
best_returns = best_positions.shift(1) * returns
best_returns = best_returns.dropna()

# Plot cumulative returns
plt.figure(figsize=(15, 10))

plt.subplot(2, 1, 1)
plt.plot(best_returns.cumsum(), label=f'Strategy Returns (Holding Period = {best_hp})')
plt.title(f'Cumulative Returns of Best CMMA-Filtered Strategy')
plt.legend()
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(squid_vwap, label='VWAP')
plt.axhline(y=FAIR_PRICE, color='r', linestyle='--', label='Fair Price (2000)')

# Plot buy and sell signals
buy_signals = best_positions.diff() > 0
sell_signals = best_positions.diff() < 0

plt.scatter(buy_signals[buy_signals].index, squid_vwap[buy_signals], 
            marker='^', s=100, color='green', label='Buy Signal (Below 2000)')
plt.scatter(sell_signals[sell_signals].index, squid_vwap[sell_signals], 
            marker='v', s=100, color='red', label='Sell Signal (Above 2000)')

plt.title('Price with Trading Signals')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 7. Conclusion

In this notebook, we've explored the impact of different holding periods on a CMMA-filtered mean reversion strategy for Squid_Ink. The strategy only buys below 2000 when CMMA is low (weak momentum) and only sells above 2000 when CMMA is high (strong momentum).

Key findings:
1. The optimal holding period is [to be filled after running]
2. Longer holding periods generally [increase/decrease] the total return but [increase/decrease] the number of trades
3. The win rate [increases/decreases] with longer holding periods
4. The Sharpe ratio is maximized at a holding period of [to be filled after running]

These findings suggest that the holding period is a critical parameter for the CMMA-filtered mean reversion strategy. The optimal holding period balances the trade-off between capturing the full mean reversion effect and minimizing exposure to adverse price movements.